In [ ]:
%matplotlib inline

import os

import mne
import numpy as np

## Timeseries data in MNE - the `Raw` class

The most basic form of electrophysiological data is timeseries data: a continuous set of voltage values recorded over time for each [channel](https://mne.tools/stable/documentation/glossary.html#term-channels).

Timeseries data in MNE is stored in [`mne.io.Raw`](https://mne.tools/stable/generated/mne.io.Raw.html) and [`mne.io.RawArray`](https://mne.tools/stable/generated/mne.io.RawArray.html) objects.

`Raw` objects can be created through loading data from the disk via one of the various [`mne.io.read_raw_xxx()`](https://mne.tools/stable/api/reading_raw_data.html) functions.

`RawArray` objects can be created from data [arrays](https://numpy.org/doc/stable/reference/arrays.html) directly.

### Part 1 - Reading timeseries data from disk

To familiarise ourselves with `Raw` objects, we will start by loading [MNE's sample dataset](https://mne.tools/stable/documentation/datasets.html#sample).

In [ ]:
# Filepath to MNE's sample dataset on disk
sample_data_folder = mne.datasets.sample.data_path()

# Load sample data from disk
raw = mne.io.read_raw_fif(
    os.path.join(sample_data_folder, "MEG", "sample", "sample_audvis_raw.fif")
)

`Raw` objects contain:
- the timeseries data
- the metadata, stored under the `info` attribute

In [ ]:
# Show information about the data object
raw.info

For example, here you can see that the data we have loaded contains a mixture of MEG data (gradiometers and magnetometers) and EEG data sampled at 600 Hz, as well as the timings of stimulus presentation and subject behaviour during the recording ([stimulus channels](https://mne.tools/stable/documentation/glossary.html#term-stim-channel)).

The data itself can be accessed via the [`get_data()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.get_data) method, which returns an array of shape `(channels, times)`.

In [ ]:
# Get data as an array
data = raw.get_data()
print(f"Data has shape: {data.shape} (channels, times)")

`Raw` objects have various methods for working with timeseries data, such as:
- isolating specific channels - [`pick()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.pick), [`drop_channels()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.drop_channels)
- isolating specific windows of time - [`crop()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.crop)
- spectral filtering of the data - [`filter()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.filter), [`notch_filter()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.notch_filter)
- plotting the data - [`plot()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.plot)
- computing the power spectra of the data - [`compute_psd()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.compute_psd)

We will explore some of these methods below, and others in later notebooks.

**Exercises - Manipulating `Raw` objects**

We will start by selecting only a subset of channels to store in our `Raw` object, using the [`pick()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.pick) method.

`pick()` accepts channel names, channel types, or channel indices as input, and retains only those channels that match this criteria.

Below, we select only the MEG channels.

*Hint:* Generally in MNE, methods will modify the object in-place to save memory. Because we want to play around with the data without modifying the original object, we will first make a copy of the `Raw` object using the [`copy()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.copy) method.

In [ ]:
# Create a copy of the data so that we can manipulate it
raw_copy = raw.copy()

# Select the MEG channels only
raw_copy.pick("meg")

# Verify that we have only MEG channels
print(raw_copy.get_data().shape)
raw_copy.info

As you can see, the new `Raw` object now has only 306 channels, corresponding to the 203 Gradiometers and 102 Magnetometers (as well as 1 'bad' channel where the data was not properly recorded).

**Exercise:** Select only the EEG channels from the original `Raw` object, and verify that only these channels remain.

In [ ]:
# Remember to copy the original data!
raw_copy = raw.copy()

## CODE GOES HERE
raw_copy.pick("eeg")
print(raw_copy.get_data().shape)
raw_copy.info

**Exercise:** Now select the EEG and MEG channels simultaneously, and verify that only these channels remain.

In [ ]:
# Remember to copy the original data
raw_copy = raw.copy()

## CODE GOES HERE
raw_copy.pick(["eeg", "meg"])
print(raw_copy.get_data().shape)
raw_copy.info


**Exercise:** Select the EEG and stimulus channels simultaneously, and verify that only these channels remain.

In [ ]:
raw_copy = raw.copy()

## CODE GOES HERE
raw_copy.pick("stim")
print(raw_copy.get_data().shape)
raw_copy.info

**Exercise:** Select three channels of your choice by specifying their names, and verify that only these channels remain.

*Hint:* channel names are stored in the `ch_names` attribute of the `Raw` object.

In [ ]:
raw_copy = raw.copy()

## CODE GOES HERE
raw_copy.pick(raw_copy.ch_names[:3])
print(raw_copy.get_data().shape)
print(raw_copy.ch_names)
raw_copy.info

To proceed, we will select only the EEG data from the original `Raw` object.

Having isolated the EEG data, we can visualise it using the [`plot()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.plot) method.

Navigate through the different channels using the up and down arrow keys, and navigate through the timepoints using the left and right arrow keys.

The home key reduces the time window displayed, and the end key increases the time window displayed.

In [ ]:
# Select EEG data
raw_eeg = raw.copy().pick("eeg")

# Plot EEG data
raw_eeg.plot(scalings="auto");

We can also plot the locations of the EEG sensors using the [`plot_sensors()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.plot_sensors) method.

Why is one of the channels red?

In [ ]:
%matplotlib inline

# Plot EEG sensors
raw_eeg.plot_sensors(show_names=True);

%matplotlib widget

If you go to the end of the recording, you will see that is has a duration of ~280 seconds.

You can find the exact end time using the `times` attribute of the `Raw` object, which contains the times of each sample in the data.

In [ ]:
# Show how many timepoints are in the data
print(f"{raw_eeg.times.shape[0]} timepoints in the data")

# Take the last timepoint to show the duration of the data
print(f"{raw_eeg.times[-1] :.0f} seconds of data")

Just like we were able to select only a specific set of channels, we can also select only a specific window of time using the [`crop()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.crop) method.

Below, we omit the first 10 seconds of the recording (as for `pick()`, this modifies the object in-place).

In [ ]:
# Units of time should be in seconds
raw_eeg.crop(tmin=10, tmax=None)

**Exercise:** Verify that the duration of the recording has been reduced by 10 seconds by plotting the data or using the number of timepoints.

In [ ]:
## CODE GOES HERE
print(f"{raw_eeg.times[-1] :.0f} seconds of data")

**Exercise:** Omit the last 10 seconds of the same object, and verify that the duration of the recording has been reduced by a further 10 seconds.

In [ ]:
## CODE GOES HERE
duration = raw_eeg.times[-1]
raw_eeg.crop(tmin=0, tmax=duration - 10)
print(f"{raw_eeg.times[-1] :.0f} seconds of data")

**Exercise:** Select only the time between 30 and 60 seconds, and verify that the duration of the recording is now 30 seconds.

In [ ]:
## CODE GOES HERE
raw_eeg.crop(tmin=30, tmax=60)
print(f"{raw_eeg.times[-1] :.0f} seconds of data")
raw_eeg.plot();

With this brief overview of `Raw` objects, we have seen how timeseries data can loaded and manipulated in MNE.

Generally, timeseries data is loaded from disk using one of the many `mne.read_raw_xxx()` functions tailored to specific data formats, like we have done above.

See also the [MNE-BIDS](https://mne.tools/mne-bids/stable/index.html) package for loading data in the BIDS format.

However, it is sometimes also useful to create `Raw` objects from arrays directly, which we will explore below.

### Part 2 - Creating `RawArray` objects from data arrays

Rather than using `Raw` objects to store data from arrays, we must instead use the [`RawArray`](https://mne.tools/stable/generated/mne.io.RawArray.html) class.

But first, this requires having some data to store!

Below we randomly generate some data consisting of 3 channels and 1,000 timepoints. Remember, MNE expects timeseries data to have shape `(channels, times)`.

In [ ]:
# Define parameters for generating data
n_channels = 3
n_times = 1000  # samples
np.random.seed(44)  # set seed for consistency

# Generate the data
data = np.random.randn(n_channels, n_times)
print(f"Data has shape: {data.shape} (channels, times)")

If we want to store this data in a `RawArray` object, we need to also specify the metadata, so that MNE can keep track of what the data represents.

This information is stored as an [`mne.Info`](https://mne.tools/stable/generated/mne.Info.html) object, which we create using the [`mne.create_info()`](https://mne.tools/stable/generated/mne.create_info.html) function.

For this, we need to specify:
- the names of the channels - `ch_names` parameter
- the types of the channels - `ch_types` parameter
- the sampling frequency - `sfreq` parameter

In [ ]:
# Create the data information
info = mne.create_info(ch_names=["CH_1", "CH_2", "CH_3"], ch_types="eeg", sfreq=100)

# Show what is stored in the information object
print(f"Channel names: {info['ch_names']}")
info

As you can see, we have created an `Info` object for 3 EEG channels. We arbitrarily set the sampling frequency to 100 Hz (with 1,000 samples, this corresponds to 10 seconds of data).

**Exercises - Creating `RawArray` objects from arrays**

We can also specify the types of each channel separately using the `ch_types` argument.

**Exercise:** Create an `Info` object for 3 EEG channels, specifying the type of each channel separately.

In [ ]:
## CODE GOES HERE
info = mne.create_info(ch_names=["CH_1", "CH_2", "CH_3"], ch_types=["eeg", "eeg", "eeg"], sfreq=100)
info

**Exercise:** Create an `Info` object for 3 channels, where the first is EEG, the second a gradiometer, and the third a magnetometer.

In [ ]:
## CODE GOES HERE
info = mne.create_info(
    ch_names=["CH_1", "CH_2", "CH_3"], ch_types=["eeg", "grad", "mag"], sfreq=100
)
info

As you may have noticed, specific bits of information can be accessed from the `Info` object in the same way you would access information from a [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries).

**Exercise:** Get the sampling frequency from the `Info` object.

In [ ]:
## CODE GOES HERE
info["sfreq"]

**Exercise:** Get the channel names from the `Info` object.

In [ ]:
## CODE GOES HERE
info["ch_names"]

Using an `Info` object where all 3 channels are EEG, we can now create a `RawArray` object.

In [ ]:
# Create data information
info = mne.create_info(ch_names=["CH_1", "CH_2", "CH_3"], ch_types="eeg", sfreq=100)

# Store the data and information in a RawArray object
raw = mne.io.RawArray(data=data, info=info)

# Show what is stored in the RawArray object
raw.info

Although the `RawArray` object is of a different class to the `Raw` object we were working with before, is still supports the same methods, e.g. [`pick()`](https://mne.tools/stable/generated/mne.io.RawArray.html#mne.io.RawArray.pick), [`crop()`](https://mne.tools/stable/generated/mne.io.RawArray.html#mne.io.RawArray.crop), [`plot()`](https://mne.tools/stable/generated/mne.io.RawArray.html#mne.io.RawArray.plot), etc...

Here we again use [`plot()`](https://mne.tools/stable/generated/mne.io.RawArray.html#mne.io.RawArray.plot) to visualise the data.

In [ ]:
raw.plot(scalings="auto");

The `Info` object we created is now stored under the `info` attribute of the `RawArray` object.

**Exercise:** Get the sampling frequency from the `info` attribute of the `RawArray` object.

In [ ]:
## CODE GOES HERE
raw.info["sfreq"]

**Exercise:** Get the channel names from the `info` attribute of the `RawArray` object.

In [ ]:
## CODE GOES HERE
raw.info["ch_names"]

## Conclusion

`Raw` class objects are one of the most heavily used items in the MNE package, being the way in which timeseries data is stored.

They can be created easily from data stored on the disk (`mne.read_raw_xxx()` -> `Raw`), or from arrays (`array` -> `RawArray`).

Here we have covered the very basic aspects of handling timeseries data in MNE. In later notebooks, we will explore the more advanced features of `Raw` objects, including filtering activity in particular frequency ranges, and computing the power spectral densities of data.

## Additional resources

MNE tutorial on `Raw` objects: https://mne.tools/stable/auto_tutorials/raw/10_raw_overview.html